In [1]:
%pip install -qU langchain langchain-openai langchain-anthropic langsmith python-dotenv pydantic

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

from langsmith import Client

client = Client()


In [3]:
example_system_prompt = """You are a helpful assistant

Rules:
- You are only allowed to talk about coding
- <HERE>
- Try to be concise"""

example_instruction = "The agent should only respond in English. No other languages"

examples = ["Hola, como estas?", "Hi there!"]

In [4]:
from typing import List
import random
from langsmith.schemas import Run, Example
from langsmith import evaluate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.chat_models import init_chat_model
from schemas import Rule, Rules, CorrectnessEvaluationResult


class Promptnado:
    def __init__(self, system_prompt: str, instruction: str, examples: List[str], rule_token="<HERE>",
                 rule_gen_model=init_chat_model("gpt-4o-mini", temperature=0.7),
                 eval_model=init_chat_model("gpt-4o-mini", temperature=0.7),
                 prediction_model=init_chat_model("gpt-4o-mini", temperature=0.7)):
        self.system_prompt = system_prompt
        self.instruction = instruction
        self.examples = examples
        self.rule_token = rule_token

        # Create random dataset name
        self.dataset_name = f"Promptnado_{random.randint(0, 1000000)}"

        self.attempts = 1
        self.solved = False
        self.current_rule = None
        self.current_prompt = None
        self.successful_prompt = None
        self.rule_gen_model = rule_gen_model
        self.eval_model = eval_model
        self.prediction_model = prediction_model

    def _create_dataset(self):
        """Create a dataset with a unique name"""
        dataset = client.create_dataset(
            self.dataset_name, description=self.instruction)
        for example in self.examples:
            client.create_example(
                inputs={"input": example}, dataset_id=dataset.id)

        self.dataset = dataset
        print(
            f"Created dataset: {self.dataset_name} with {len(self.examples)} examples")
        return dataset

    def _generate_rules(self):
        """Use an LLM to generate a list of rules"""

        system_prompt = f"""You are an expert LLM Prompt Engineer. Your job is to try to solve for the provided <Instructions> \
by making adjustments to the <Original Prompt>. You should attempt to make 5 suggestions for prompts that might work. Each suggestion you \
make will be interpolated into the prompt where {self.rule_token} is, and then evaluated for correctness against a dataset of \
examples.

<Instructions>
{self.instruction}
</Instructions>

<Original Prompt>
{self.system_prompt}
</Original Prompt>
"""

        structured_llm = self.rule_gen_model.with_structured_output(Rules)

        rules: Rules = structured_llm.invoke(system_prompt)

        self.rules = rules.rules
        print(f"Generated {len(self.rules)} rules\n")
        print(self.rules)
        return self.rules

    def _build_prompt(self, rule: Rule):
        """Interpolate the rules into the system prompt"""
        interpolated_prompt = self.system_prompt.replace(
            self.rule_token, rule.prompt)

        print(f"Interpolated prompt:\n\n{interpolated_prompt}")
        return interpolated_prompt

    def _evaluate_correctness(self, run: Run, example: Example):
        """Eval function to use an LLM to validate that the instruction was followed"""
        system_prompt = f"""Your job is to validate whether the <Result> meets the criteria for <Instruction>. Try to be a harsh judge.

<Instruction>
{self.instruction}
</Instruction>

<Result>
{run.outputs["output"]}
</Result>
"""

        structured_llm = self.eval_model.with_structured_output(
            CorrectnessEvaluationResult)

        result: CorrectnessEvaluationResult = structured_llm.invoke(
            system_prompt)

        return {"score": 1 if result.correct else 0, "key": "correctness", "comment": result.reasoning}

    def _predict(self, inputs: dict):
        """Run current prompt against example in the dataset"""
        try:
            messages = [
                SystemMessage(content=self.current_prompt),
                HumanMessage(content=inputs["input"]),
            ]

            # Invoke the model
            response = self.prediction_model.invoke(messages)
            return {"output": response.content}

        except Exception as e:
            print(f"Error predicting: {e}")
            raise e

    def _is_solved(self, eval_results):
        """Validate the results"""

        results = eval_results._results

        # If any of the result scores are not a 1, return false
        if len(results) == 0:
            raise Exception("No results found")

        for result in results:
            score = result['evaluation_results']["results"][0].score
            if score != 1:
                return False

        return True

    def _test_rule(self, rule: Rule):
        """Evaluate a given rule"""
        print(f'\nTesting rule: "{rule.prompt}"')
        self.current_rule = rule

        self.current_prompt = self._build_prompt(self.current_rule)

        results = evaluate(
            self._predict,
            data=self.dataset_name,
            evaluators=[self._evaluate_correctness],
            experiment_prefix=f"Attempt-{self.attempts}",
        )

        self.attempts += 1

        return results

    def run(self):
        """Run the promptnado"""
        print(f"Running Promptnado with instruction: {self.instruction}")
        # Create the dataset
        self._create_dataset()

        while self.solved == False:
            try:
                self._generate_rules()
                for rule in self.rules:
                    results = self._test_rule(rule)
                    if self._is_solved(results):
                        self.results = results
                        self.solved = True
                        self.successful_prompt = self.current_prompt
                        break
            except Exception as e:
                print(f"Fatal error encountered: {e}")
                break  # Exit the while loop on error

        print("\n\nSolved!! Current prompt can be found at `self.successful_prompt`\n\n")

        print(
            f"Successful prompt:\n====================\n{self.current_prompt}\n=================")
        print(self.results._manager._experiment.url)

/Users/camron/Documents/Dev/promptnado/.venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `init_chat_model` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [5]:
pn = Promptnado(example_system_prompt, example_instruction, examples)
pn.run()

Running Promptnado with instruction: The agent should only respond in English. No other languages
Created dataset: Promptnado_320453 with 2 examples
Generated 5 rules

[Rule(reasoning='This prompt directly specifies the language limitation, making it clear that the assistant should respond only in English.', prompt='You must respond only in English and no other languages.'), Rule(reasoning="By stating the requirement for English responses upfront, it sets a clear expectation for the assistant's language use.", prompt='Please ensure all responses are in English, without exceptions.'), Rule(reasoning='This version emphasizes the exclusive use of English, reinforcing the requirement throughout the conversation.', prompt='All responses should exclusively be in English; do not use any other language.'), Rule(reasoning="This prompt highlights the importance of adhering to the language rule, ensuring that it is a priority in the assistant's responses.", prompt='You are required to communicate

/Users/camron/Documents/Dev/promptnado/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'Attempt-1-652a82a4' at:
https://smith.langchain.com/o/d967989d-4221-53db-b0a5-665b504acba2/datasets/54d87e20-565a-4a13-b229-abbfc6cfc045/compare?selectedSessions=cdc49ac3-393a-4eed-9873-bc29bf0e9413




2it [00:03,  1.72s/it]



Solved!! Current prompt can be found at `self.successful_prompt`


Successful prompt:
You are a helpful assistant

Rules:
- You are only allowed to talk about coding
- You must respond only in English and no other languages.
- Try to be concise
https://smith.langchain.com/o/d967989d-4221-53db-b0a5-665b504acba2/projects/p/cdc49ac3-393a-4eed-9873-bc29bf0e9413


In [6]:
# Keys in results
keys = pn.results._manager._experiment.url
keys

'https://smith.langchain.com/o/d967989d-4221-53db-b0a5-665b504acba2/projects/p/cdc49ac3-393a-4eed-9873-bc29bf0e9413'